# RBP Maps 0.2 can be used as a reference for SE events
- This notebook verifies the changes made do not affect the last time dataframes were checked independently. 

In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import sys
import math

sys.path.insert(0, '/home/bay001/projects/codebase/rbp-maps/maps/')
from pandas.util.testing import assert_frame_equal
from density import ReadDensity
from density import normalization_functions as norm
from density import intervals
from density import Feature
pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 1500)

In [2]:
wd = '/home/bay001/projects/codebase/temp/'
outdir = '/home/bay001/projects/codebase/temp/out/'
ref_wd = '/home/bay001/projects/maps_20160420/analysis/tests/for_eric/'
read_nums = '/home/bay001/projects/codebase/data/ALLDATASETS_submittedonly.txt.mapped_read_num'
ipbam = '/home/bay001/projects/codebase/temp/204_01_RBFOX2.merged.r2.bam'
inpbam = '/home/bay001/projects/codebase/temp/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam'
ref_positive = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-positive.txt'
ref_negative = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-negative.txt'
ref_bg = '/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE.txt'
program = '/home/bay001/projects/codebase/rbp-maps/maps/plot_density.py'
readnums = pd.read_table(read_nums,names=['rbp','reads'])


In [3]:
def make():
    ref1 = pd.read_table('/projects/ps-yeolab3/bay001/maps/annotations/RBFOX2-HepG2-SE.txt')
    ref1 = ref1[ref1['IncLevelDifference']>0.1]
    ref1.to_csv('/home/bay001/projects/codebase/temp/RBFOX2-HepG2-SE-positive.txt',sep='\t',index=None) 

def diff(df1, df2):
    try:
        assert_frame_equal(df1, df2)
        return('same')
    except Exception as e:  # appeantly AssertionError doesn't catch all
        return('{} different'.format(e))
def round_nearest(x, a):
    return round(round(x / a) * a, -int(math.floor(math.log10(a))))


In [4]:
! python $program -ip $ipbam -input $inpbam -c $ref_positive $ref_negative $ref_bg -at rmats -o $outdir -norm 1 -e SE

all files found, skipping norm.bw creation.
/home/bay001/projects/codebase/temp/204_01_RBFOX2.merged.r2.bam
/home/bay001/projects/codebase/temp/RBFOX2-204-INPUT_S2_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam

Creating SE RBP Map
finished creating matrix
label: RBFOX2-HepG2-SE-positive
output base: /home/bay001/projects/codebase/temp/out/204_01_RBFOX2.merged.r2
feature: feature
finished normalizing
finished setting means

Creating SE RBP Map
finished creating matrix
label: RBFOX2-HepG2-SE-negative
output base: /home/bay001/projects/codebase/temp/out/204_01_RBFOX2.merged.r2
feature: feature
finished normalizing
finished setting means

Creating SE RBP Map
finished creating matrix
label: RBFOX2-HepG2-SE
output base: /home/bay001/projects/codebase/temp/out/204_01_RBFOX2.merged.r2
feature: feature
finished normalizing
finished setting means
Found 3 features to plot.
range() integer end argument expected, got numpy.float64.
/home/bay001/anaconda/lib/python2.7/site-packages/matp

### Test input raw matrix

In [5]:
ref_input_raw_matrix = 'included.input.204_01_RBFOX2.included.feature.se.raw_density_matrix.csv'
ref_input = pd.read_table(
    os.path.join(ref_wd,ref_input_raw_matrix),
    sep=',',
    index_col=0
)
test_input_raw_matrix = '/home/bay001/projects/codebase/temp/out/204_01_RBFOX2.merged.r2.input.se.RBFOX2-HepG2-SE-positive.feature.se.raw_density_matrix.csv'
test_input = pd.read_table(test_input_raw_matrix,sep=',',
    index_col=0)

diff(test_input, ref_input)

'same'

### Test ip raw matrix

In [6]:
# test ip
ref_ip_raw_matrix = 'included.ip.204_01_RBFOX2.included.feature.se.raw_density_matrix.csv'
ref_ip = pd.read_table(
    os.path.join(ref_wd,ref_ip_raw_matrix),
    sep=',',
    index_col=0
)
test_ip_raw_matrix = '/home/bay001/projects/codebase/temp/out/204_01_RBFOX2.merged.r2.ip.se.RBFOX2-HepG2-SE-positive.feature.se.raw_density_matrix.csv'
test_ip = pd.read_table(test_ip_raw_matrix,sep=',',
    index_col=0)

diff(test_ip, ref_ip)

'same'

# test ip pdf function

In [7]:
test_ip_pdf = norm.calculate_pdf(test_ip)
ref_ip_pdf = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/ip_pdf.csv',sep=',',index_col=0)
diff(test_ip_pdf,ref_ip_pdf)

'same'

# test input pdf function

In [8]:
test_input_pdf = norm.calculate_pdf(test_input)
ref_input_pdf = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/input_pdf.csv',sep=',',index_col=0)
diff(test_input_pdf,ref_input_pdf)

'same'

# Test normalization (subtract by region) function
- caveats: see below

In [9]:

# ip reads: 4275451
# input reads: 3765762


test_norm = norm.normalize_and_per_region_subtract(test_ip, test_input,1000000.0/4275451,1000000.0/3765762,0)
ref_norm = pd.read_table('/home/bay001/projects/maps_20160420/analysis/tests/for_eric/included.204_01_RBFOX2.subtract_by_region.feature.normed_matrix.csv',
                        sep=',',index_col=0)

merged = pd.merge(ref_norm, test_norm, how='left', left_index=True, right_index=True)

test_norm = test_norm.ix[merged.index]
test_norm.columns = test_norm.columns + '_TEST'

ref_norm = ref_norm.ix[merged.index]
ref_norm.columns = ref_norm.columns + '_REF'

print(test_norm.shape)
print(ref_norm.shape)

(2698, 1400)
(2698, 1400)


In [10]:
merged = pd.merge(ref_norm, test_norm, how='left', left_index=True, right_index=True)
merged.shape

(2698, 2800)

##### diff(test_norm, ref_norm)
- This will likely return false due to the rounding issues. When the ref was written to file, it rounded off to the nearest 14th digit. However the normalization function will not round at all, so we need to manually do this to check for equality. 
- Somehow, the reference was truncated. ARGH!
- but the good news is that it is all matching up to the truncation. 

In [11]:
merged.head()

,0_REF,1_REF,2_REF,3_REF,4_REF,5_REF,6_REF,7_REF,8_REF,9_REF,...,1390_TEST,1391_TEST,1392_TEST,1393_TEST,1394_TEST,1395_TEST,1396_TEST,1397_TEST,1398_TEST,1399_TEST
"10003\tENSG00000090539.11\tCHRD\tchr3\t+\t184105161\t184105200\t184104838\t184104895\t184105718\t184105821\t10003\t46,21\t26,0\t16,1\t4,3\t138\t100\t0.360867390803\t1.0\t0.562,1.0\t0.743,0.195\t0.312",0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,0.000121,-0.000048,-0.000048,0.000242,...,0.001864,0.001864,0.001864,0.002155,0.002155,0.002155,0.002155,0.002155,0.002155,0.002155
"10004\tENSG00000090539.11\tCHRD\tchr3\t+\t184104279\t184104543\t184103833\t184103947\t184104632\t184104726\t10004\t28,15\t2,4\t37,2\t6,3\t199\t100\t0.362396362008\t1.0\t0.876,0.653\t0.756,0.251\t0.261",-0.000078,-0.000191,-0.000191,-0.000191,-0.000191,-0.000191,-0.000078,-0.000496,-0.000496,-0.000496,...,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748,-0.001748
"10007\tENSG00000090539.11\tCHRD\tchr3\t+\t184105161\t184105265\t184104838\t184104895\t184105718\t184105821\t10007\t104,33\t26,0\t37,7\t4,3\t199\t100\t0.564126921431\t1.0\t0.668,1.0\t0.823,0.54\t0.153",0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,0.000125,-0.000051,-0.000051,0.000250,...,0.001931,0.001931,0.001931,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232,0.002232
"10012\tENSG00000111271.10\tACAD10\tchr12\t+\t112183993\t112184088\t112182446\t112182847\t112184852\t112185012\t10012\t15,10\t0,0\t12,4\t2,0\t194\t100\t1.0\t1.0\t1.0,1.0\t0.756,1.0\t0.122",-0.000087,-0.000087,-0.000087,-0.000087,-0.000087,0.000447,0.000447,-0.000665,-0.000665,-0.000087,...,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044,-0.000044
"10027\tENSG00000254363.2\tCTB-131B5.5\tchr5\t+\t139548221\t139548400\t139547265\t139547387\t139552332\t139552636\t10027\t0,2\t0,0\t7,6\t0,1\t199\t100\t1.0\t1.0\tNA,1.0\t1.0,0.751\t0.125",0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004


In [12]:

for c in range(0,1400):
    for i in merged.index:
        if i != ix_in_question: # we don't need to check the index that was truncated. 
            t = round_nearest(merged.ix[i]['{}_TEST'.format(c)],13)
            r = round_nearest(merged.ix[i]['{}_REF'.format(c)],13)
            if(r != t):
                if np.isnan(r) and np.isnan(r):
                    pass
                else:
                    print(i, round_nearest(
                            merged.ix[i]['{}_TEST'.format(col)],13), 
                          round_nearest(
                            merged.ix[i]['{}_REF'.format(col)],13))


NameError: name 'ix_in_question' is not defined

# Test better 5p function

In [ ]:
original == better

In [ ]:
original